Другой пример использования объектов для инкапсуляции некоторой "ответственности" — [менеджеры контекста](https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers) в Python. Мы с ними уже виделись — это конструкция `with ...:`, которая позволяет, например, закрывать файл после использования:

In [1]:
with open('nul', 'r') as f:  # Или /dev/null, если у вас Linux
    data = f.read()
    assert not data

На самом деле никакой магии здесь не происходит. `open` возвращайт объект, который является не только файлом, но и _менеджером контекста_. Это, в терминах Python, объект, у которого есть два метода: `__enter__` (вызывается при входе в блок `with`) и `__exit__` (вызывается при выходе из блока любым способом — и завершение блока, и `return`, и выброс исключение).

In [2]:
from contextlib import AbstractContextManager

class ContextManagerExample(AbstractContextManager):
    def __enter__(self):
        print('Entering with')
        return 123  # Будет записано конструкцией `... as x`
    
    def __exit__(self, exc_type, exc_value, traceback):
        print('Exiting with')
        # Значения, которые описывают исключение, которое вылетело в блоке
        # (или None, если он завершился сам)
        print('exc_type:', exc_type)
        print('exc_value:', exc_value)
        print('traceback:', traceback)

In [3]:
manager = ContextManagerExample()
print('Manager created')
with manager as x:
    print('x is {}'.format(x))
print('Block finished')

Manager created
Entering with
x is 123
Exiting with
exc_type: None
exc_value: None
traceback: None
Block finished


In [4]:
print('Before block')
with manager as x:
    print('x is {}'.format(x))
    raise NotImplementedError('hello')
print('Block finished')

Before block
Entering with
x is 123
Exiting with
exc_type: <class 'NotImplementedError'>
exc_value: hello
traceback: <traceback object at 0x03F4C508>


NotImplementedError: hello

Обратите внимание на две особенности:

1. Метод `__exit__` вызывается в любом случае, даже при выбросе исключения. Более того: нам дают всю информацию про это исключение (`traceback` содержит информацию о том, где конкретно было выброшено исключение) и мы можем в зависимости от этого менять своё поведение.
2. Создание менеджера контекста и вход/выход в блок `with` — это независимые операции. Конструкция `with open(...) as f` — это лишь внезапно оказавшаяся удобной форма записи, поэтому файл сделали менеджером контекста, который в `__enter__` не делает ничего (и так уже открыт), а в `__exit__` закрывает файл. Поэтому файл — не очень хороший менеджер контекста, его нельзя переиспользовать. Зато можно писать красиво: `with open(...) as f`.

Этот пример показывает, что объекты можно использовать и даже если у нас вообще никаких данных нет, только логика, которую мы хотим куда-то передать.

Другой пример менеджера контекста, который вы уже видели — `pytest.raises`. Он в `__exit__` проверяет, что было выброшено определённое исключение.

В качестве ещё одного примера напишем менеджер контекста, который замеряет время выполнения блока:

In [5]:
import time

class Timer:
    def __enter__(self):
        self.start_time = time.process_time()
        
    def __exit__(self, exc_type, exc_value, traceback):
        self.end_time = time.process_time()
        self.duration = self.end_time - self.start_time

In [6]:
t = Timer()
with t:
    [2 * x for x in range(int(1e6))]
print(t.duration)

with t:
    [2 * x for x in range(int(1e7))]
print(t.duration)

0.12480079999999993
1.0920069999999997


Дальше можно было бы добавить в `Timer` возможность запомнить все проведённые измерения, посчитать среднее время выполнения, ещё что-нибудь... Но мы этим заниматься не будем.

Основная мысль та же, что и в прошлом файле: объект характеризуется тем, за что он отвечает. В прошлом файле отвечал за чтение данных (байт, сообщений, чего угодно), в этом файле отвечает за то, что выполняется "вокруг" некоторого куска кода.

In [7]:
# Кстати, есть синтаксический сахар, позволяющий писать менеджеры контекста попороще: при помощи декораторов и yield
import time
from contextlib import contextmanager

@contextmanager
def time_and_print():
    start_time = time.process_time()
    yield  # В этот момент `time_and_print` прерывается, а декоратор может передать управление в блок with
    print(time.process_time() - start_time)

In [8]:
with time_and_print():
    [2 * x for x in range(int(1e6))]
with time_and_print():
    [2 * x for x in range(int(1e7))]

0.1092006999999997
1.1232072


По сути `@contextmanager` берёт произвольный генератор и собирает из него менеджер контекста: в `__enter__` выполняется код до первого `yield` (т.е. до первого "элемента", который генератор возвращает), а в `__exit__` — всё остальное.